In [1]:
import laspy
import open3d as o3d
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
import sys, os
sys.path.append(os.path.abspath(os.path.join('../scripts')))

In [46]:
pl = PythonLidar(epsg=3857)

2022-06-24 23:36:50,477 — FileHandler — ERROR — usgs_3dep_metadata not found
Traceback (most recent call last):
  File "c:\Users\user\10-Accademy\W7\USGS_3DEP_Python_LIDAR\scripts\file_handler.py", line 49, in read_csv
    path = logger.DATAS_PATH / str(name + '.csv')
  File "c:\Users\user\.conda\envs\pdalpy_n\lib\site-packages\pandas\util\_decorators.py", line 311, in wrapper
    return func(*args, **kwargs)
  File "c:\Users\user\.conda\envs\pdalpy_n\lib\site-packages\pandas\io\parsers\readers.py", line 680, in read_csv
    return _read(filepath_or_buffer, kwds)
  File "c:\Users\user\.conda\envs\pdalpy_n\lib\site-packages\pandas\io\parsers\readers.py", line 575, in _read
    parser = TextFileReader(filepath_or_buffer, **kwds)
  File "c:\Users\user\.conda\envs\pdalpy_n\lib\site-packages\pandas\io\parsers\readers.py", line 933, in __init__
    self._engine = self._make_engine(f, self.engine)
  File "c:\Users\user\.conda\envs\pdalpy_n\lib\site-packages\pandas\io\parsers\readers.py", line

In [3]:
las = laspy.read('../data/ept-data/10-0-833-513.laz')
las

<LasData(1.2, point fmt: <PointFormat(1, 5 bytes of extra dims)>, 27029 points, 3 vlrs)>

In [5]:
las.header
las.header.point_format
las.header.point_count
las.vlrs

[<ExtraBytesVlr(extra bytes structs: 2)>, <GeoKeyDirectoryVlr(7 geo_keys)>, <GeoAsciiParamsVlr(['WGS 84 / Pseudo-Mercator|WGS 84|', ''])>]

In [6]:
list(las.point_format.dimension_names)

['X',
 'Y',
 'Z',
 'intensity',
 'return_number',
 'number_of_returns',
 'scan_direction_flag',
 'edge_of_flight_line',
 'classification',
 'synthetic',
 'key_point',
 'withheld',
 'scan_angle_rank',
 'user_data',
 'point_source_id',
 'gps_time',
 'ClassFlags',
 'OriginId']

In [11]:
print('x=',  las.X)
print('intensity=', las.intensity)
print('coordinate =', las.gps_time)

x= [-6834730 -6834747 -6834762 ... -6839794 -6839812 -6839794]
intensity= [314 324 269 ... 144 249 290]
coordinate = [1.99824630e+08 1.99824630e+08 1.99824630e+08 ... 1.99826321e+08
 1.99826321e+08 1.99826321e+08]


In [14]:
# classes
classes = set(list(las.classification))

In [15]:
point_data = np.stack([las.X, las.Y, las.Z], axis=0).transpose((1, 0))
point_data

array([[-6834730,  4292758,    14563],
       [-6834747,  4292656,    14568],
       [-6834762,  4292561,    14575],
       ...,
       [-6839794,  4305274,    13845],
       [-6839812,  4305185,    13832],
       [-6839794,  4305864,    13837]])

In [16]:
ground = laspy.create(point_format=las.header.point_format, file_version=las.header.version)
ground.points = las.points[las.classification == 2]

ImportError: cannot import name 'JVisualizer' from 'open3d' (c:\Users\user\.conda\envs\pdalpy_n\lib\site-packages\open3d\__init__.py)

In [25]:
geom = o3d.geometry.PointCloud()
geom.points = o3d.utility.Vector3dVector(point_data)
o3d.visualization.draw_geometries([geom])

[Open3D WARNING] [ViewControl] SetViewPoint() failed because window height and width are not set.
